In [25]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [26]:
house_dict = []

for page in range(16, 21):
    linked = 'https://www.zillow.com/ca/duplex/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-130.93471906436687,"east":-107.64370343936687,"south":31.980790552461904,"north":48.0418909706512},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false},"isCondo":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:58.0) Gecko/20100101 Firefox/58.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw")[1].text
            except:
                zestimate = 'null'
            try:
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 bloUvX")[2].text
                
            except:
                rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

https://www.zillow.com/homedetails/7351-La-Mesita-Pl-La-Mesa-CA-91942/295355798_zpid/
zestimate $959,006
rent zestimate $2,295/mo
https://www.zillow.com/homedetails/675-River-Rd-San-Miguel-CA-93451/122245576_zpid/
zestimate $341,981
rent zestimate $2,340/mo
https://www.zillow.com/homedetails/29705-Avenida-La-Paz-B-Cathedral-City-CA-92234/2076769877_zpid/
zestimate $650,001
rent zestimate null
https://www.zillow.com/homedetails/3344-Lakeshore-Blvd-Lakeport-CA-95453/2077200911_zpid/
zestimate $822,674
rent zestimate null
https://www.zillow.com/homedetails/384-Walnut-Ave-Long-Beach-CA-90802/21238385_zpid/
zestimate $775,108
rent zestimate $2,400/mo
https://www.zillow.com/homedetails/1001-Columbia-St-Santa-Cruz-CA-95060/16105035_zpid/
zestimate $2,032,486
rent zestimate $4,500/mo
https://www.zillow.com/homedetails/531-58th-St-Oakland-CA-94609/24753897_zpid/
zestimate $1,472,858
rent zestimate $3,100/mo
https://www.zillow.com/homedetails/1007-W-University-Ave-Fresno-CA-93705/18740083_zpid/


zestimate $664,840
rent zestimate $2,700/mo
https://www.zillow.com/homedetails/2202-Hillcrest-Dr-Los-Angeles-CA-90016/20595850_zpid/
zestimate $1,400,006
rent zestimate $2,600/mo
https://www.zillow.com/homedetails/1427-Balhan-Dr-Concord-CA-94521/2077956691_zpid/
zestimate $825,005
rent zestimate null
https://www.zillow.com/homedetails/60-Marina-Court-Dr-San-Rafael-CA-94901/19248213_zpid/
zestimate $1,640,379
rent zestimate $4,000/mo
https://www.zillow.com/homedetails/210-Plaza-St-Healdsburg-CA-95448/15744497_zpid/
zestimate $862,944
rent zestimate $2,250/mo
https://www.zillow.com/homedetails/42837-Willow-Ave-Big-Bear-Lake-CA-92315/17620458_zpid/
zestimate $456,493
rent zestimate null
https://www.zillow.com/homedetails/2454-Johnston-St-Los-Angeles-CA-90031/20637496_zpid/
zestimate $1,076,821
rent zestimate $2,495/mo
https://www.zillow.com/homedetails/1833-Benecia-Ave-Los-Angeles-CA-90025/20500182_zpid/
zestimate $3,677,376
rent zestimate $3,100/mo
https://www.zillow.com/homedetails/1310

zestimate $424,650
rent zestimate $1,695/mo
https://www.zillow.com/homedetails/1224-1226-Tait-St-Oceanside-CA-92054/2077337714_zpid/
zestimate $1,765,075
rent zestimate null
https://www.zillow.com/homedetails/413-N-Harvard-Blvd-Los-Angeles-CA-90004/20781606_zpid/
zestimate $2,955,893
rent zestimate $2,900/mo
https://www.zillow.com/homedetails/458-460-35th-Ave-San-Francisco-CA-94121/2077541269_zpid/
zestimate $3,161,466
rent zestimate null
https://www.zillow.com/homedetails/2025-E-White-Ave-Fresno-CA-93701/2076887586_zpid/
zestimate $485,004
rent zestimate null
https://www.zillow.com/homedetails/1326-Bennett-Ave-Long-Beach-CA-90804/21222109_zpid/
zestimate $897,059
rent zestimate $2,271/mo
https://www.zillow.com/homedetails/4176-Dorsey-St-Los-Angeles-CA-90011/20620397_zpid/
zestimate $549,905
rent zestimate $2,250/mo
https://www.zillow.com/homedetails/9738-42-Prospect-Ave-Lakeside-CA-92040/295348035_zpid/
zestimate $955,007
rent zestimate $2,295/mo
https://www.zillow.com/homedetails/153

zestimate $1,190,003
rent zestimate $2,800/mo
https://www.zillow.com/homedetails/4756-Myrtle-St-Pico-Rivera-CA-90660/21085532_zpid/
zestimate $594,665
rent zestimate $2,250/mo
https://www.zillow.com/homedetails/187-189-Beaver-St-San-Francisco-CA-94114/2077233089_zpid/
zestimate $1,916,657
rent zestimate null
https://www.zillow.com/homedetails/4018-N-Bonita-St-Spring-Valley-CA-91977/295358752_zpid/
zestimate $950,003
rent zestimate $2,295/mo
https://www.zillow.com/homedetails/2436-Exposition-Pl-Los-Angeles-CA-90018/20585795_zpid/
zestimate $751,094
rent zestimate $2,800/mo
https://www.zillow.com/homedetails/3485-N-Marks-Ave-Fresno-CA-93722/82853869_zpid/
zestimate $383,445
rent zestimate $1,000/mo
https://www.zillow.com/homedetails/34244-Via-Santa-Rosa-Dana-Point-CA-92624/2097353572_zpid/
zestimate $743,998
rent zestimate null
https://www.zillow.com/homedetails/1776-Country-Club-Dr-A-Escondido-CA-92029/2077462627_zpid/
zestimate $800,005
rent zestimate null
https://www.zillow.com/homede

In [27]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,Multi-family home for sale,"$959,000","7351 La Mesita Pl, La Mesa, CA 91942",7351 La Mesita Pl,LaMesa,CA,91942,6,4,"3,028",null,null,Multiple Occupancy,1940,Covered,null,"$959,006","$2,295/mo",$317,https://www.zillow.com/homedetails/7351-La-Mes...
1,Multi-family home for sale,"$346,000","675 River Rd, San Miguel, CA 93451",675 River Rd,SanMiguel,CA,93451,2,2,"1,284",null,null,Multiple Occupancy,1940,"Off-street, Covered",null,"$341,981","$2,340/mo",$269,https://www.zillow.com/homedetails/675-River-R...
2,Multi-family home for sale,"$650,000","29705 Avenida La Paz #&-B, Cathedral City, CA ...",29705 Avenida La Paz #&-B,CathedralCity,CA,92234,6,4,"3,307",null,null,Multiple Occupancy,1996,"Garage - Attached, Covered",null,"$650,001",null,$197,https://www.zillow.com/homedetails/29705-Aveni...
3,Multi-family home for sale,"$849,000","3344 Lakeshore Blvd, Lakeport, CA 95453",3344 Lakeshore Blvd,Lakeport,CA,95453,15,13,"10,000",null,null,Multiple Occupancy,1900,0 spaces,null,"$822,674",null,$85,https://www.zillow.com/homedetails/3344-Lakesh...
4,Multi-family home for sale,"$775,000","384 Walnut Ave, Long Beach, CA 90802",384 Walnut Ave,LongBeach,CA,90802,4,4,"1,980",null,null,Multiple Occupancy,1922,0 spaces,null,"$775,108","$2,400/mo",$391,https://www.zillow.com/homedetails/384-Walnut-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Multi-family home for sale,"$800,000","1776 Country Club Dr #A, Escondido, CA 92029",1776 Country Club Dr #A,Escondido,CA,92029,7,4,--,null,null,Multiple Occupancy,1945,0 spaces,null,"$800,005",null,null,https://www.zillow.com/homedetails/1776-Countr...
196,Multi-family home for sale,"$1,500,000","13480 Beach Ave, Marina Del Rey, CA 90292",13480 Beach Ave,MarinaDelRey,CA,90292,3,--,"1,633",null,null,Multiple Occupancy,1953,"Off-street, Covered",null,"$1,501,950","$3,400/mo",$919,https://www.zillow.com/homedetails/13480-Beach...
197,Multi-family home for sale,"$380,000","3330 E Normal Ave, Fresno, CA 93703",3330 E Normal Ave,Fresno,CA,93703,4,4,"2,500",null,null,Multiple Occupancy,1951,"Garage, Garage - Attached, Covered",null,"$381,467","$1,100/mo",$152,https://www.zillow.com/homedetails/3330-E-Norm...
198,Multi-family home for sale,"$625,000","3337 City Terrace Dr, Los Angeles, CA 90063",3337 City Terrace Dr,LosAngeles,CA,90063,4,3,"4,030",null,null,Multiple Occupancy,1926,Covered,null,"$627,977","$2,300/mo",$155,https://www.zillow.com/homedetails/3337-City-T...


In [28]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q4-2020_Multi_Family_Data_page16-20.csv", index=False, header=True)